<a href="https://colab.research.google.com/github/ritwiks9635/My_Neural_Network_Architecture/blob/main/Using_pre_trained_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Using pre-trained word embeddings**

## Introduction

In this example, we show how to train a text classification model that uses pre-trained
word embeddings.

We'll work with the Newsgroup20 dataset, a set of 20,000 message board messages
belonging to 20 different topic categories.

For the pre-trained word embeddings, we'll use
[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

In [50]:
import os

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import numpy as np
import tensorflow as tf
import tensorflow.data as tf_data
import keras
from keras import layers

## Download the Newsgroup20 data

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True)

17329808/17329808 [==============================] - 3s 0us/step


In [3]:
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "sci.space")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['comp.windows.x', 'sci.electronics', 'alt.atheism', 'misc.forsale', 'rec.sport.hockey', 'talk.religion.misc', 'comp.graphics', 'comp.os.ms-windows.misc', 'sci.space', 'comp.sys.mac.hardware', 'talk.politics.guns', 'soc.religion.christian', 'comp.sys.ibm.pc.hardware', 'talk.politics.mideast', 'sci.med', 'sci.crypt', 'rec.autos', 'rec.motorcycles', 'talk.politics.misc', 'rec.sport.baseball']
Number of files in comp.graphics: 1000
Some example filenames: ['61010', '60217', '61208', '60910', '61221']


In [4]:
print(open(data_dir / "sci.space" / "61010").read())

Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!news.intercon.com!digex.com!digex.com!not-for-mail
From: prb@access.digex.com (Pat)
Newsgroups: sci.space
Subject: Re: NAVSTAR positions
Date: 21 Apr 1993 18:23:05 -0400
Organization: Express Access Online Communications, Greenbelt, MD USA
Lines: 7
Message-ID: <1r4hg9$sgi@access.digex.net>
References: <1993Apr19.063455.26363@ericsson.se>
NNTP-Posting-Host: access.digex.net

  C-3's bird may be flaking out and expecting to die soon.

or C-3  may orbit over  major users areas, and it may be
needed to provide  redundancy on that plane  while b-4  may orbit
over hicksville, and not have muc of a user community.

pat



As you can see, there are header lines that are leaking the file's category, either
explicitly (the first line is literally the category name), or implicitly, e.g. via the
`Organization` filed. Let's get rid of the headers:

In [5]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

There's actually one category that doesn't have the expected number of files, but the
difference is small enough that the problem remains a balanced classification problem.



## Shuffle and split the data into training & validation sets

In [6]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [7]:
len(train_samples), len(val_samples)

(15998, 3999)

## Create a vocabulary index

Let's use the `TextVectorization` to index the vocabulary found in the dataset.
Later, we'll use the same layer instance to vectorize the samples.

Our layer will only consider the top 20,000 words, and will truncate or pad sequences to
be actually 200 tokens long.

In [10]:
vectorizer = layers.TextVectorization(max_tokens = 20000, output_sequence_length = 200)
text_ds = tf_data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [12]:
vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'the', 'to', 'of', 'a', 'and', 'in', 'is', 'i']

In [19]:
vectorizer([["He have cat the cat is black and dog is cute"]]).numpy()[0,:11]

array([  46,   19, 3463,    2, 3463,    8,  579,    6, 1915,    8, 7406])

In [22]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [28]:
txt = ["the", "cat", "on", "the", "mat"]
[word_index[w] for w in txt]

[2, 3463, 15, 2, 8267]

## Load pre-trained word embeddings



Let's download pre-trained GloVe embeddings (a 822M zip file).

You'll need to run the following commands:

In [29]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-12-05 14:54:43--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.76MB/s    in 3m 23s  

2023-12-05 14:58:07 (4.04 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



The archive contains text-encoded vectors of various sizes: 50-dimensional,
100-dimensional, 200-dimensional, 300-dimensional. We'll use the 100D ones.

Let's make a dict mapping words (strings) to their NumPy vector representation:

In [30]:
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [33]:
len(coefs)

100

Now, let's prepare a corresponding embedding matrix that we can use in a Keras
`Embedding` layer. It's a simple NumPy matrix where entry at index `i` is the pre-trained
vector for the word of index `i` in our `vectorizer`'s vocabulary.

In [40]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17981 words (2019 misses)


Next, we load the pre-trained word embeddings matrix into an `Embedding` layer.

Note that we set `trainable=False` so as to keep the embeddings fixed (we don't want to
update them during training).

In [43]:
embedding_layer = layers.Embedding(num_tokens, embedding_dim, trainable = False)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

In [45]:
inputs = keras.Input(shape=(None,), dtype="int32")
embedded_sequences = embedding_layer(inputs)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 100)         2000200   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         64128     
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 128)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                        

## Train the model

First, convert our list-of-strings data to NumPy arrays of integer indices. The arrays
are right-padded.

In [46]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

We use categorical crossentropy as our loss since we're doing softmax classification.
Moreover, we use `sparse_categorical_crossentropy` since our labels are integers.

In [47]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 33s 250ms/step - loss: 2.7001 - acc: 0.1330 - val_loss: 2.1454 - val_acc: 0.2491
Epoch 2/20
125/125 [==============================] - 29s 232ms/step - loss: 2.0050 - acc: 0.3017 - val_loss: 1.6471 - val_acc: 0.4246
Epoch 3/20
125/125 [==============================] - 33s 264ms/step - loss: 1.5807 - acc: 0.4529 - val_loss: 1.3427 - val_acc: 0.5396
Epoch 4/20
125/125 [==============================] - 32s 258ms/step - loss: 1.3298 - acc: 0.5424 - val_loss: 1.1808 - val_acc: 0.5959
Epoch 5/20
125/125 [==============================] - 28s 224ms/step - loss: 1.1647 - acc: 0.6054 - val_loss: 1.0985 - val_acc: 0.6342
Epoch 6/20
125/125 [==============================] - 29s 229ms/step - loss: 1.0176 - acc: 0.6523 - val_loss: 1.0555 - val_acc: 0.6369
Epoch 7/20
125/125 [==============================] - 30s 241ms/step - loss: 0.9105 - acc: 0.6843 - val_loss: 1.1734 - val_acc: 0.6369
Epoch 8/20
125/125 [==============================] - 2

## Export an end-to-end model

Now, we may want to export a `Model` object that takes as input a string of arbitrary
length, rather than a sequence of indices. It would make the model much more portable,
since you wouldn't have to worry about the input preprocessing pipeline.

Our `vectorizer` is actually a Keras layer, so it's simple:

In [51]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model(
    tf.convert_to_tensor(
        [["this message is about computer graphics and 3D modeling"]]
    )
)

print(class_names[np.argmax(probabilities[0])])

comp.graphics
